# Generating german doctor reviews with a GPT-2 model
## Fine tuning of a pretrained **Hugging Face** transfomer decoder
In this notebook we will be using a GPT-2 mdoel that was fine-tuned to synthesize doctor reviews mimiking actual patients' text comments.

A detailed description of the **German language reviews of doctors by patients 2019** dataset can be found [here](https://data.world/mc51/german-language-reviews-of-doctors-by-patients)


For this exercise, we will use the [**Hugging Face**](https://huggingface.co/) implementation of transformers for Tensorflow 2.0. Transformers provides a general architecture implementation for several state of the art models in the natural language domain.

NOTE: This notebook and its implementation is heavily influenced by the data-drive [blog post](https://data-dive.com/finetune-german-gpt2-on-tpu-transformers-tensorflow-for-text-generation-of-reviews)

In [1]:
!pip install -U transformers==4.9.2

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 31.8 MB/s 
     |████████████████████████████████| 636 kB 33.4 MB/s 
     |████████████████████████████████| 3.3 MB 34.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd

from transformers import AutoTokenizer, TFGPT2LMHeadModel

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400

## Setting up the decoder model
HuggingFace's transfomer library allows for conviniently loading  pre-configured text tokenizers and pre-trained models from local resources.

Here we will be using a tokenizer and a GPT-2 model that was pre-trained on the doctor review dataset


In [3]:
!wget -O gpt2_doctorreview_finetuned.zip https://github.com/AdvancedNLP/decoder/raw/main/gpt2_doctorreview_finetuned.zip
!unzip gpt2_doctorreview_finetuned.zip

--2021-09-17 14:56:36--  https://github.com/AdvancedNLP/decoder/raw/main/gpt2_doctorreview_finetuned.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/AdvancedNLP/decoder/main/gpt2_doctorreview_finetuned.zip [following]
--2021-09-17 14:56:36--  https://media.githubusercontent.com/media/AdvancedNLP/decoder/main/gpt2_doctorreview_finetuned.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462732204 (441M) [application/zip]
Saving to: ‘gpt2_doctorreview_finetuned.zip’

gpt2_doctorreview_f 100%[===================>] 441.29M   159MB/s    in 2.8s    

2021-09-17 14:57:07 (159 MB/s)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('gpt2_doctorreview_finetuned/tokenizer')
model = TFGPT2LMHeadModel.from_pretrained('gpt2_doctorreview_finetuned/model')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2_doctorreview_finetuned/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## Generating doctor reviews
The model has been conditioned to be able to control if positive or negative reviews should be generated. 

As an auto-regressive model the sequence is generated by building up from the passed input sequences. We can use this to control the polarity of the review by passing either the token for positive or for negative reviews

In [5]:
POS_TOKEN = "<|review_pos|>"
NEG_TOKEN = "<|review_neg|>"

#### High level pipeline
The easiest way to to use the model is to use HuggingFaces transformer `pipeline` implementation to encapsulate the previously loaded `model` and `tokenizer`.

The documentation for the [**pipeline**](https://huggingface.co/transformers/main_classes/pipelines.html) abstraction describes how to do the setup


In [6]:
from transformers import pipeline

In [7]:
##########################
## YOUR CODE HERE START ##
##########################
# build a transformer-pipeline 
# to generate text using the 
# previously loaded model and tokenizer

review_generator = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
)

##########################
## YOUR CODE HERE END   ##
##########################

In [8]:
pos_generated_reviews = review_generator(POS_TOKEN, max_length=150, num_return_sequences=6)
pd.DataFrame(pos_generated_reviews)

,generated_text
0,"<|review_pos|> Freund an der Rezeption, obwohl die Praxis immer schon geschlossen hat und die Leute anrufe die nicht weg geschickt werden"
1,<|review_pos|> Wir haben uns bei Fr. Dr. Jansen gut behandelt gefühlt.
2,<|review_pos|> Als Neupatientin wurde ich im Vorfeld nicht über den Ablauf der Untersuchung informiert. Die Ärztin nahm sich viel Zeit und hat mich über die Untersuchung und die Behandlungsempfehlung umfassend beraten. Ich würde mich jederzeit wieder in diese Praxis begeben. Das Team ist auch sehr freundlich.
3,<|review_pos|> Ich fühle mich in der Praxis und bei Zahnärzten immer sehr gut aufgehoben. Auch die Prophylaxebehandlung ist so angenehm wie es in den meisten Praxen passiert. Die Behandlung ist stets sehr professionell und kompetent. Sehr zu empfehlen!
4,<|review_pos|> Bin seit mehr als Jahren bei Hr. Eissa und bin immer noch vollsten zufrieden mit seiner Behandlung! Er nimmt sich viel Zeit ist sehr kompetent und man kann ihn nur weiterempfehlen
5,"<|review_pos|> Ich konnte meine Angst durch die Hilfe von Hrn. Dr. Herdt nehmen. Der Zahnarzt nimmt Zeit, erklärt alles, fragt nach, hört sich die Bedürfnisse des Patienten an und erklärt ausführlich die notwendige Behandlungen. In dieser Praxis hat man einfach das Gefühl, daß man in wirklich guten Händen ist. Ein ganz herzliches Dankeschön an Hrn. Dr. Herdt für die hervorragende Behandlung und Betreuung."


In [9]:
neg_generated_reviews = review_generator(NEG_TOKEN, max_length=150, num_return_sequences=6)
pd.DataFrame(neg_generated_reviews)

,generated_text
0,"<|review_neg|> Leider wird es hier keine Notwendigkeit geben, auf die Patienten einzugehen und das in einer für den Patienten sehr unangenehmen Praxisumgebung. Sehr distanzierte Praxisführung. Leider nur Tage auf den Kontrolltermin gewartet, daher eine Wartezeit von Stunden. Dies trifft jedoch in den nachfolgenden Behandlungsräumen zu. Ich hatte das Gefühl, mir wurde unnötig teure Leistungen an der Rezeption empfohlen, die nichts mit der jeweiligen Behandlung zu tun haben. Die Diagnose stimmte bisher nicht."
1,"<|review_neg|> Die Praxis ist sehr schön und modern eingerichtet. Nachdem ich im Behandlungszimmer platz nehmen musste, habe ich mir sofort die Praxis ausgesucht. Ich musste keine min warten bei anderen Ärzten. Ich bin dann bei Dr. Behle reingegangen und sie hat sich sehr wenig Zeit genommen, hat kaum bis kaum zugehört und hat mir dann eine falsche Behandlung angeboten. Außerdem habe ich die Behandlung abgebrochen und kam um ein paar Tage später wieder zu ihr, da sie einfach die anderen Termine hatte. Ich werde nur noch dahin gehen."
2,<|review_neg|> Ich war dort weil mein Allgemeinmediziner vor Monaten seine Praxis geschlossen hatte. Man hatte bei Frau Dr. Karabulut angefangen von ihrem Praxiskollegen an den Tag zu legen Sie ging nicht auf meine Beschwerden ein und versuchte die Untersuchung vorzunehmen. Dies war dann . Auf meine Fragen was denn nun das Thema sei sagte sie es sei alles in Ordnung nur zu dieser Person. Ich würde nicht noch einmal hingehen.
3,"<|review_neg|> Dr. Bader machte auf mich einen kompetenten Eindruck ich hätte ihn darauf aufmerksam gemacht das er an einem Samstag zur Sprechstunde und um Uhr einen anderen Dienst hätte nehmen können. Auch nach einer Röntgenaufnahme, die wegen der Entzündung meines Armes nicht erfolgte, wurde nicht auf Beschwerden eingegangen und die Röntgenbilder wurden auch nicht gemacht. Mein Fazit Die Empfangsdame war etwas anderes als freundlich bis vor die Ärztin. Die Wartezeit war völlig in Ordnung aber trotz Termin über eine Stunde sind die Wartezeiten nicht zu ertragen."
4,"<|review_neg|> Wir waren zur Vorsorgeuntersuchung da und das mit großem Abstand am frühen Morgen. Am Schluss hat die Assistenzärztin leider kein gutes und gesundes Auge und das auch noch auf eine Kontrolle. Das ging so dann los dass alles sehr schnell ging. Wir hatten ein Gespräch mit dem Arzt während der Untersuchung und das erste was am Telefon erfragte wie es ihm ging und warum er etwas gefunden hat. Die Untersuchung wurde mir von der Ärztin mitgeteilt, dann wird der Befund nicht wahrgenommen! Sie will nicht wissen, wie er denn von ihrem Arzt zu erwarten wäre und dann würde er die Mutte..."
5,"<|review_neg|> Der erste Versuch hat angefangen, aber Herr Dr. Lammertz war nicht besonders einfühlsam. Er war sehr arrogant, hat kein Interesse daran zu reden, sondern nur einfach rein aus dem Behandlungszimmer zu fragen. Er wirkte so als würde man sich einen Arztbesuch in Zukunft sparen, er nimmt sich keine Zeit für den Patienten. Er ist völlig genervt und hat eine sehr agressive Art an sich. Zu dem sehr sympatisch! Sehr, sehr schade und schade!"


### Low level control over text generation
So far so good but sometimes you will need more control over how the text is being generated.



#### Greedy search
The following code can be used to generate text using a greedy search algorithm:

In [10]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(POS_TOKEN, return_tensors='tf')

# generate text until the output length
# (which includes the context length) reaches 50 
greedy_outputs = model.generate(
    input_ids, 
    max_length=50,
    num_return_sequences=3,
    )

genrated_reviews = [{'generated_text': tokenizer.decode(output, skip_special_tokens=True)}
                    for output in greedy_outputs]
pd.DataFrame(genrated_reviews)

,generated_text
0,"Ich hatte vor einem halben Jahr eine größere Sanierung, nach einem Jahr schmerzloser Behandlung kann jetzt schon ein Teil der Krone in,, abgeschlossen werden. Dr. Peter Wilck, dentscherer, kann ich nur für die hervorragende Leistung empfehlen."
1,Ich war schon mehrfach bei Dr. Hillenbrand und bin äußerst zufrieden. Er nimmt sich Zeit und ist auf alle Belange und Bedürfnisse sehr freundlich eingegangen. Dr. Hillenbrand hat sich auch ausreichend Zeit für Beratung und Untersuchung genommen und einen kompetenten
2,Ich bin heute auf die Hilfe von Frau Dr. Schulz gestoßen und bin seit ca. einem Jahr Patient in der Facharztklinik. Ich fühlte mich dort sehr gut aufgehoben und kann die Praxis empfehlen.


#### Beam search 
Beam search can be considered as an alternative. At each step of generating a token, a set of top probability tokens are kept as part of the beam instead of just the highest-probability token. The sequence with the highest overall probability is returned at the end of the generation.

What do the parameters `no_repeat_ngram_size` and `temperature` control?

Generating text using beam search is done like this:

In [11]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=7,
    no_repeat_ngram_size=3,
    num_return_sequences=3,
    early_stopping=True,
    temperature=0.7
)

genrated_reviews = [{'generated_text': tokenizer.decode(output, skip_special_tokens=True)}
                    for output in beam_outputs]
pd.DataFrame(genrated_reviews)

,generated_text
0,Ich bin seit Jahren bei Frau Dr. Henze und fühle mich bei ihr sehr gut aufgehoben. Sie ist sehr kompetent und nimmt sich Zeit für ihre Patienten. Ich kann sie nur weiterempfehlen.
1,Ich bin seit Jahren bei Frau Dr. Henze und fühle mich bei ihr sehr gut aufgehoben. Sie ist sehr kompetent und nimmt sich Zeit für ihre Patienten. Ich kann sie nur weiterempfehlen.
2,Ich bin seit Jahren bei Frau Dr. Henze und fühle mich bei ihr sehr gut aufgehoben. Sie ist sehr kompetent und nimmt sich Zeit für ihre Patienten. Ich kann sie nur weiterempfehlen.
